<H2>CHAPTER 5 - Lab: Cross-Validation and the Bootstrap</H2>

In this lab, we explore the resampling techniques covered in this chapter. Some of the commands in this lab may take a while to run on your
computer.
We again begin by placing most of our imports at this top level.

In [3]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
summarize,
poly)
from sklearn.model_selection import train_test_split


There are several new imports needed for this lab.

In [5]:
 from functools import partial
from sklearn.model_selection import \
(cross_validate,
KFold,
ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

<h4>The Validation Set Approach</h4>

We use the function train_test_split() to split the data into training  and validation sets. As there are 392 observations, we split into two equal
sets of size 196 using the argument test_size=196. It is generally a good
idea to set a random seed when performing operations like this that contain
an element of randomness, so that the results obtained can be reproduced
precisely at a later time. We set the random seed of the splitter with the
argument random_state=0

In [8]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
test_size=196,
random_state=0)

Now we can fit a linear regression using only the observations corresponding to the training set Auto_train.

In [10]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

We now use the predict() method of results evaluated on the model matrix for this model created using the validation data set. We also calculate
the validation MSE of our model.

In [13]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

np.float64(23.616617069669882)

Hence our estimate for the validation MSE of the linear regression fit is
23.62.
We can also estimate the validation error for higher-degree polynomial
regressions. We first provide a function evalMSE() that takes a model string
as well as a training and test set and returns the MSE on the test set.

In [17]:
def evalMSE(terms,
            response,
            train,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)


Let’s use this function to estimate the validation MSE using linear,
quadratic and cubic fits. We use the enumerate() function here, which gives enumerate() both the values and indices of objects as one iterates over a for loop.

In [20]:
# Initialize an array to store MSE for degrees 1, 2, 3
MSE = np.zeros(3)

# Loop through polynomial degrees 1, 2, 3
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid)

# Display the MSE array
MSE


array([23.61661707, 18.76303135, 18.79694163])

These error rates are 23.62, 18.76, and 18.80, respectively. If we choose a
different training/validation split instead

In [23]:
# Split dataset into training and validation sets
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)

# Initialize array to store MSE for polynomial degrees 1, 2, 3
MSE = np.zeros(3)

# Loop over polynomial degrees
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                        'mpg',
                        Auto_train,
                        Auto_valid)

# Display the MSE array
MSE


array([20.75540796, 16.94510676, 16.97437833])

The class sklearn_sm() has as its first argument a model from statsmodels.
It can take two additional optional arguments: model_str which can be used
to specify a formula, and model_args which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic
regression model we have to specify a family argument. This is passed as
model_args={'family':sm.families.Binomial()}.

In [25]:
 hp_model = sklearn_sm(sm.OLS,
MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
X,
Y,
cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

np.float64(24.231513517929233)

The arguments to cross_validate() are as follows: an object with the appropriate fit(), predict(), and score() methods, an array of features X and
a response Y. We also included an additional argument cv to cross_validate();
specifying an integer K results in K-fold cross-validation. We have provided
a value corresponding to the total number of observations, which results
in leave-one-out cross-validation (LOOCV). The cross_validate() func- cross_
tion produces a dictionary with several components; we simply want the validate()
cross-validated test score here (MSE), which is estimated to be 24.23.
We can repeat this procedure for increasingly complex polynomial fits.
To automate the process, we again use a for loop which iteratively fits
polynomial regressions of degree 1 to 5, computes the associated crossvalidation error, and stores it in the ith element of the vector cv_error.
The variable d in the for loop corresponds to the degree of the polynomial.
We begin by initializing the vector. This command may take a couple of
seconds to run.

In [29]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
Y = np.array(Auto['mpg'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1, 6)):
    # Design matrix for polynomial regression
    X = np.column_stack([H**p for p in range(d+1)])
    
    # LOOCV with MSE
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0],                # LOOCV
                          scoring='neg_mean_squared_error',
                          return_train_score=False)
    
    # Store MSE (convert negative to positive)
    cv_error[i] = -np.mean(M_CV['test_score'])

cv_error


array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.0332262 ])

Above we introduced the outer() method of the np.power() function. .outer()
np.power() The outer() method is applied to an operation that has two arguments,
such as add(), min(), or power(). It has two arrays as arguments, and then
forms a larger array where the operation is applied to each pair of elements
of the two arrays.

In [31]:
 A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In the CV example above, we used K = n, but of course we can also use
K<n. The code is very similar to the above (and is significantly faster).
Here we use KFold() to partition the data into K = 10 random groups. We KFold()
use random_state to set a random seed and initialize a vector cv_error in
which we will store the CV errors corresponding to the polynomial fits of
degrees one to five.

In [34]:
cv_error = np.zeros(5)

# Use the same K-Fold splits for all degrees
cv = KFold(n_splits=10, shuffle=True, random_state=0)

for i, d in enumerate(range(1, 6)):
    # Create polynomial design matrix for degree d
    X = np.power.outer(H, np.arange(d+1))
    
    # Perform cross-validation
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    
    # Store mean test score
    cv_error[i] = np.mean(M_CV['test_score'])

# Display cross-validation errors
cv_error


array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13719075])

The cross_validate() function is flexible and can take different splitting
mechanisms as an argument. For instance, one can use the funtion to implement the validation set approach just as easily as K-fold
cross-validation.

In [36]:
validation = ShuffleSplit(n_splits=1,
test_size=196,
random_state=0)
results = cross_validate(hp_model,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation);
results['test_score']

array([23.61661707])

One can estimate the variability in the test error by running the following:

In [38]:
validation = ShuffleSplit(n_splits=10,
test_size=196,
random_state=0)
results = cross_validate(hp_model,
Auto.drop(['mpg'], axis=1),
Auto['mpg'],
cv=validation)
results['test_score'].mean(), results['test_score'].std()

(np.float64(23.802232661034164), np.float64(1.4218450941091862))

<h4>Estimating the Accuracy of a Statistic of Interest</h4>

We will create a function alpha_func(), which takes as input a dataframe D
assumed to have columns X and Y, as well as a vector idx indicating which
observations should be used to estimate α. The function then outputs the
estimate for α based on the selected observations.

In [42]:
Portfolio = load_data('Portfolio')

def alpha_func(D, idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ( (cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]) )


This function returns an estimate for α based on applying the minimum
variance formula (5.7) to the observations indexed by the argument idx. For
instance, the following command estimates α using all 100 observations.

In [44]:
alpha_func(Portfolio, range(100))

np.float64(0.57583207459283)

Next we randomly select 100 observations from range(100), with replacement. This is equivalent to constructing a new bootstrap data set and
recomputing αˆ based on the new data set.

In [46]:
 rng = np.random.default_rng(0)
alpha_func(Portfolio,
rng.choice(100,
100,
replace=True))

np.float64(0.6074452469619004)

This process can be generalized to create a simple function boot_SE() for
computing the bootstrap standard error for arbitrary functions that take
only a data frame as an argumen

In [50]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)


Notice the use of _ as a loop variable in for _ in range(B). This is often
used if the value of the counter is unimportant and simply makes sure the
loop is executed B times.
Let’s use our function to evaluate the accuracy of our estimate of α using
B = 1,000 bootstrap replications.

In [52]:
alpha_SE = boot_SE(alpha_func,
Portfolio,
B=1000,
seed=0)
alpha_SE

np.float64(0.09118176521277699)

<h4>Estimating the Accuracy of a Linear Regression Model</h4>

We start by writing a generic function boot_OLS() for bootstrapping a
regression model that takes a formula to define the corresponding regression. We use the clone() function to make a copy of the formula that can clone() be refit to the new dataframe. This means that any derived features such
as those defined by poly() (which we will see shortly), will be re-fit on the
resampled data frame.

In [89]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.iloc[idx]  # Use iloc to index by integer positions
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [91]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [92]:
rng = np.random.default_rng(0)

np.array([
    hp_func(Auto, rng.choice(Auto.shape[0], Auto.shape[0], replace=True))
    for _ in range(10)
])


array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

Next, we use the boot_SE() function to compute the standard errors of
1,000 bootstrap estimates for the intercept and slope terms.

In [72]:
hp_se = boot_SE(hp_func,
Auto,
B=1000,
seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

This indicates that the bootstrap estimate for SE(βˆ0) is 0.85, and that
the bootstrap estimate for SE(βˆ1) is 0.0074. As discussed in Section 3.1.2,
standard formulas can be used to compute the standard errors for the
regression coefficients in a linear model. These can be obtained using the
summarize() function from ISLP.sm.

In [74]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

Below we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model
to the data. Since this model provides a good fit to the data (Figure 3.8),
there is now a better correspondence between the bootstrap estimates and
the standard estimates of SE(βˆ0), SE(βˆ1) and SE(βˆ2).

In [77]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
quad_model,
'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

We compare the results to the standard errors computed using sm.OLS().

In [78]:
M = sm.OLS(Auto['mpg'],
quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64